## Feast Client with RBAC
### RBAC Kubernetes Authentication
Feast **Role-Based Access Control (RBAC)** in Kubernetes supports authentication  **inside a Kubernetes pod** and **outside a pod** when running a local script.
### Inside a Kubernetes Pod
Feast automatically retrieves the Kubernetes ServiceAccount token from:
```
/var/run/secrets/kubernetes.io/serviceaccount/token
```
This means:
- No manual configuration is needed inside a pod.
- The token is mounted automatically and used for authentication.
- Developer just need create the binding with role and service account accordingly.
- Code Reference:  
[Feast Kubernetes Auth Client Manager (Pod Token Usage)](https://github.com/feast-dev/feast/blob/master/sdk/python/feast/permissions/client/kubernetes_auth_client_manager.py#L15) 
- See the example will use service account from pod [Example](https://github.com/feast-dev/feast/blob/master/examples/rbac-remote/client/k8s/)

### Outside a Kubernetes Pod (Local Machine)
If running Feast outside of Kubernetes, authentication requires setting the token manually:
```sh
export LOCAL_K8S_TOKEN="your-service-account-token"
```
Feast will use this token for authentication.

Reference:  
[Feast Authentication via `LOCAL_K8S_TOKEN`](https://github.com/feast-dev/feast/blob/master/sdk/python/feast/permissions/client/kubernetes_auth_client_manager.py#L50)

## Test Cases
| User Type       | ServiceAccount               | RoleBinding Assigned | Expected Behavior in output                                |
|----------------|-----------------------------|----------------------|------------------------------------------------------------|
| **Read-Only**  | `feast-user-sa`              | `feast-reader`       | Can **read** from the feature store, but **cannot write**. |
| **Unauthorized** | `feast-unauthorized-user-sa` | _None_               | **Access should be denied** in `test.py`.                  |
| **Admin**      | `feast-admin-sa`             | `feast-writer`       | Can **read and write** feature store data.                 |

###  Feature Store settings

In [7]:
!cat client/feature_store.yaml

project: feast_rbac
provider: local
offline_store:
    host: localhost
    type: remote
    port: 8081
online_store:
    path: http://localhost:8082
    type: remote
registry:
    path: localhost:8083
    registry_type: remote
auth:
    type: kubernetes
entity_key_serialization_version: 3


**The Operator creates the client ConfigMap containing the feature_store.yaml. We can retrieve it and port froward to local**

In [8]:
!kubectl get configmap feast-sample-kubernetes-auth-client -n feast -o jsonpath='{.data.feature_store\.yaml}' 

project: feast_rbac
provider: local
offline_store:
    host: feast-sample-kubernetes-auth-offline.feast.svc.cluster.local
    type: remote
    port: 80
online_store:
    path: http://feast-sample-kubernetes-auth-online.feast.svc.cluster.local:80
    type: remote
registry:
    path: feast-sample-kubernetes-auth-registry.feast.svc.cluster.local:80
    registry_type: remote
auth:
    type: kubernetes
entity_key_serialization_version: 3


### The function below is executed to support the preparation of client testing.

Run Port Forwarding for All Services for local testing 

In [19]:
import subprocess

# Define services and their local ports
services = {
    "offline_store": ("feast-sample-kubernetes-auth-offline", 8081),
    "online_store": ("feast-sample-kubernetes-auth-online", 8082),
    "registry": ("feast-sample-kubernetes-auth-registry", 8083),
}

# Start port-forwarding for each service
port_forward_processes = {}
for name, (service, local_port) in services.items():
    cmd = f"kubectl port-forward svc/{service} -n feast {local_port}:80"
    process = subprocess.Popen(cmd, shell=True)
    port_forward_processes[name] = process
    print(f"Port forwarding {service} -> localhost:{local_port}")

Port forwarding feast-sample-kubernetes-auth-offline -> localhost:8081
Port forwarding feast-sample-kubernetes-auth-online -> localhost:8082
Port forwarding feast-sample-kubernetes-auth-registry -> localhost:8083


Function to retrieve a Kubernetes service account token and set it as an environment variable

In [10]:
import subprocess
import os

def get_k8s_token(service_account):
    namespace = "feast"

    if not service_account:
        raise ValueError("Service account name is required.")

    result = subprocess.run(
        ["kubectl", "create", "token", service_account, "-n", namespace],
        capture_output=True, text=True, check=True
    )

    token = result.stdout.strip()

    if not token:
        return None  # Silently return None if token retrieval fails

    os.environ["LOCAL_K8S_TOKEN"] = token
    return "Token Retrieved: ***** (hidden for security)"


**Generating training data. The following test functions were copied from the `test_workflow.py` template but we added `try` blocks to print only 
the relevant error messages, since we expect to receive errors from the permission enforcement modules.**

In [11]:
from feast import FeatureStore

def fetch_historical_features_entity_df(store: FeatureStore, for_batch_scoring: bool):
    try:
        entity_df = pd.DataFrame.from_dict(
            {
                "driver_id": [1001, 1002, 1003],
                "event_timestamp": [
                    datetime(2021, 4, 12, 10, 59, 42),
                    datetime(2021, 4, 12, 8, 12, 10),
                    datetime(2021, 4, 12, 16, 40, 26),
                ],
                "label_driver_reported_satisfaction": [1, 5, 3],
                # values we're using for an on-demand transformation
                "val_to_add": [1, 2, 3],
                "val_to_add_2": [10, 20, 30],
            }
        )
        if for_batch_scoring:
            entity_df["event_timestamp"] = pd.to_datetime("now", utc=True)

        training_df = store.get_historical_features(
            entity_df=entity_df,
            features=[
                "driver_hourly_stats:conv_rate",
                "driver_hourly_stats:acc_rate",
                "driver_hourly_stats:avg_daily_trips",
                "transformed_conv_rate:conv_rate_plus_val1",
                "transformed_conv_rate:conv_rate_plus_val2",
            ],

        ).to_df()
        print(training_df.head())

    except Exception as e:
        print(f"An error occurred while fetching historical features: {e}")


def fetch_online_features(store, source: str = ""):
    try:
        entity_rows = [
            # {join_key: entity_value}
            {
                "driver_id": 1001,
                "val_to_add": 1000,
                "val_to_add_2": 2000,
            },
            {
                "driver_id": 1002,
                "val_to_add": 1001,
                "val_to_add_2": 2002,
            },
        ]
        if source == "feature_service":
            features_to_fetch = store.get_feature_service("driver_activity_v1")
        elif source == "push":
            features_to_fetch = store.get_feature_service("driver_activity_v3")
        else:
            features_to_fetch = [
                "driver_hourly_stats:acc_rate",
                "transformed_conv_rate:conv_rate_plus_val1",
                "transformed_conv_rate:conv_rate_plus_val2",
            ]
        returned_features = store.get_online_features(
            features=features_to_fetch,
            entity_rows=entity_rows,
        ).to_dict()
        for key, value in sorted(returned_features.items()):
            print(key, " : ", value)

    except Exception as e:
        print(f"An error occurred while fetching online features: {e}")

### Test Read-Only Feast User 
**Step 1: Set the Token**

In [12]:
get_k8s_token("feast-user-sa")

'Token Retrieved: ***** (hidden for security)'

**Step 2: Test misc functions from offline, online, materialize_incremental, and others**

In [20]:
!echo "Running the Feast RBAC test for a Read only User..."
from feast.data_source import PushMode
from feast import FeatureStore
from datetime import datetime
import pandas as pd

try:

    store = FeatureStore(repo_path="client")

    print("\n--- Historical features for training ---")
    fetch_historical_features_entity_df(store, for_batch_scoring=False)

    print("\n--- Historical features for batch scoring ---")
    fetch_historical_features_entity_df(store, for_batch_scoring=True)

    try:
        print("\n--- Load features into online store/materialize_incremental ---")
        feature_views= store.list_feature_views()
        if not feature_views:
            raise PermissionError("No access to feature-views or no feature-views available.")
        store.materialize_incremental(end_date=datetime.now())
    except PermissionError as pe:
        print(f"Permission error: {pe}")
    except Exception as e:
        print(f"An occurred while performing materialize incremental: {e}")

    print("\n--- Online features ---")
    fetch_online_features(store)

    print("\n--- Online features retrieved (instead) through a feature service---")
    fetch_online_features(store, source="feature_service")

    print(
        "\n--- Online features retrieved (using feature service v3, which uses a feature view with a push source---"
    )
    fetch_online_features(store, source="push")

    print("\n--- Simulate a stream event ingestion of the hourly stats df ---")
    event_df = pd.DataFrame.from_dict(
        {
            "driver_id": [1001],
            "event_timestamp": [datetime.now()],
            "created": [datetime.now()],
            "conv_rate": [1.0],
            "acc_rate": [1.0],
            "avg_daily_trips": [1000],
        }
    )
    store.push("driver_stats_push_source", event_df, to=PushMode.ONLINE_AND_OFFLINE)

    print("\n--- Online features again with updated values from a stream push---")
    fetch_online_features(store, source="push")

except Exception as e:
    print(f"An error occurred: {e}")


Running Feast RBAC test for Read only User...

--- Historical features for training ---
Handling connection for 8083
Handling connection for 8081
   driver_id           event_timestamp  label_driver_reported_satisfaction  \
0       1001 2021-04-12 10:59:42+00:00                                   1   
1       1002 2021-04-12 08:12:10+00:00                                   5   
2       1003 2021-04-12 16:40:26+00:00                                   3   

   val_to_add  val_to_add_2  conv_rate  acc_rate  avg_daily_trips  \
0           1            10   0.568053  0.225828              614   
1           2            20   0.449785  0.860379              686   
2           3            30   0.153563  0.300467              725   

   conv_rate_plus_val1  conv_rate_plus_val2  
0             1.568053            10.568053  
1             2.449785            20.449785  
2             3.153563            30.153563  

--- Historical features for batch scoring ---
Handling connection for 8081
   d

  0%|                                                                         | 0/5 [00:00<?, ?it/s]


An occurred while performing materialize incremental: 

--- Online features ---
Handling connection for 8082
acc_rate  :  [None, None]
conv_rate_plus_val1  :  [None, None]
conv_rate_plus_val2  :  [None, None]
driver_id  :  [1001, 1002]

--- Online features retrieved (instead) through a feature service---
Handling connection for 8082
conv_rate  :  [None, None]
conv_rate_plus_val1  :  [None, None]
conv_rate_plus_val2  :  [None, None]
driver_id  :  [1001, 1002]

--- Online features retrieved (using feature service v3, which uses a feature view with a push source---
Handling connection for 8082
acc_rate  :  [None, None]
avg_daily_trips  :  [None, None]
conv_rate  :  [None, None]
conv_rate_plus_val1  :  [None, None]
conv_rate_plus_val2  :  [None, None]
driver_id  :  [1001, 1002]

--- Simulate a stream event ingestion of the hourly stats df ---
An error occurred: 


### Test Unauthorized Feast User 

In [21]:
# Retrieve and store the token
get_k8s_token("feast-unauthorized-user-sa")

'Token Retrieved: ***** (hidden for security)'

In [22]:
!echo "Running Feast RBAC test for Unauthorized User..."
from feast.data_source import PushMode
from feast import FeatureStore
from datetime import datetime
import pandas as pd

try:

    store = FeatureStore(repo_path="client")

    print("\n--- Historical features for training ---")
    fetch_historical_features_entity_df(store, for_batch_scoring=False)

    print("\n--- Historical features for batch scoring ---")
    fetch_historical_features_entity_df(store, for_batch_scoring=True)

    try:
        print("\n--- Load features into online store/materialize_incremental ---")
        feature_views= store.list_feature_views()
        if not feature_views:
            raise PermissionError("No access to feature-views or no feature-views available.")
        store.materialize_incremental(end_date=datetime.now())
    except PermissionError as pe:
        print(f"Permission error: {pe}")
    except Exception as e:
        print(f"An occurred while performing materialize incremental: {e}")

    print("\n--- Online features ---")
    fetch_online_features(store)

    print("\n--- Online features retrieved (instead) through a feature service---")
    fetch_online_features(store, source="feature_service")

    print(
        "\n--- Online features retrieved (using feature service v3, which uses a feature view with a push source---"
    )
    fetch_online_features(store, source="push")

    print("\n--- Simulate a stream event ingestion of the hourly stats df ---")
    event_df = pd.DataFrame.from_dict(
        {
            "driver_id": [1001],
            "event_timestamp": [datetime.now()],
            "created": [datetime.now()],
            "conv_rate": [1.0],
            "acc_rate": [1.0],
            "avg_daily_trips": [1000],
        }
    )
    store.push("driver_stats_push_source", event_df, to=PushMode.ONLINE_AND_OFFLINE)

    print("\n--- Online features again with updated values from a stream push---")
    fetch_online_features(store, source="push")

except Exception as e:
    print(f"An error occurred: {e}")


Running Feast RBAC test for Unauthorized User...

--- Historical features for training ---
An error occurred while fetching historical features: Permission error:
Permission feast_admin_permission denied execution of ['DESCRIBE'] to FeatureView:driver_hourly_stats: Requires roles ['feast-writer'],Permission feast_user_permission denied execution of ['DESCRIBE'] to FeatureView:driver_hourly_stats: Requires roles ['feast-reader']

--- Historical features for batch scoring ---
An error occurred while fetching historical features: Permission error:
Permission feast_admin_permission denied execution of ['DESCRIBE'] to FeatureView:driver_hourly_stats: Requires roles ['feast-writer'],Permission feast_user_permission denied execution of ['DESCRIBE'] to FeatureView:driver_hourly_stats: Requires roles ['feast-reader']

--- Load features into online store/materialize_incremental ---
Permission error: No access to feature-views or no feature-views available.

--- Online features ---
An error occur

## Test Admin Feast User

In [23]:
# Retrieve and store the token
get_k8s_token("feast-admin-sa")

'Token Retrieved: ***** (hidden for security)'

In [24]:
!echo "Running Feast RBAC test for Admin User..."
from feast.data_source import PushMode
from feast import FeatureStore
from datetime import datetime
import pandas as pd

try:

    store = FeatureStore(repo_path="client")

    print("\n--- Historical features for training ---")
    fetch_historical_features_entity_df(store, for_batch_scoring=False)

    print("\n--- Historical features for batch scoring ---")
    fetch_historical_features_entity_df(store, for_batch_scoring=True)

    try:
        print("\n--- Load features into online store/materialize_incremental ---")
        feature_views= store.list_feature_views()
        if not feature_views:
            raise PermissionError("No access to feature-views or no feature-views available.")
        store.materialize_incremental(end_date=datetime.now())
    except PermissionError as pe:
        print(f"Permission error: {pe}")
    except Exception as e:
        print(f"An occurred while performing materialize incremental: {e}")

    print("\n--- Online features ---")
    fetch_online_features(store)

    print("\n--- Online features retrieved (instead) through a feature service---")
    fetch_online_features(store, source="feature_service")

    print(
        "\n--- Online features retrieved (using feature service v3, which uses a feature view with a push source---"
    )
    fetch_online_features(store, source="push")

    print("\n--- Simulate a stream event ingestion of the hourly stats df ---")
    event_df = pd.DataFrame.from_dict(
        {
            "driver_id": [1001],
            "event_timestamp": [datetime.now()],
            "created": [datetime.now()],
            "conv_rate": [1.0],
            "acc_rate": [1.0],
            "avg_daily_trips": [1000],
        }
    )
    store.push("driver_stats_push_source", event_df, to=PushMode.ONLINE_AND_OFFLINE)

    print("\n--- Online features again with updated values from a stream push---")
    fetch_online_features(store, source="push")

except Exception as e:
    print(f"An error occurred: {e}")


Running Feast RBAC test for Admin User...

--- Historical features for training ---
Handling connection for 8081
   driver_id           event_timestamp  label_driver_reported_satisfaction  \
0       1001 2021-04-12 10:59:42+00:00                                   1   
1       1002 2021-04-12 08:12:10+00:00                                   5   
2       1003 2021-04-12 16:40:26+00:00                                   3   

   val_to_add  val_to_add_2  conv_rate  acc_rate  avg_daily_trips  \
0           1            10   0.568053  0.225828              614   
1           2            20   0.449785  0.860379              686   
2           3            30   0.153563  0.300467              725   

   conv_rate_plus_val1  conv_rate_plus_val2  
0             1.568053            10.568053  
1             2.449785            20.449785  
2             3.153563            30.153563  

--- Historical features for batch scoring ---
Handling connection for 8081
   driver_id                  event_t

  0%|                                                                         | 0/5 [00:00<?, ?it/s]


An occurred while performing materialize incremental: 

--- Online features ---
Handling connection for 8082
acc_rate  :  [None, None]
conv_rate_plus_val1  :  [None, None]
conv_rate_plus_val2  :  [None, None]
driver_id  :  [1001, 1002]

--- Online features retrieved (instead) through a feature service---
Handling connection for 8082
conv_rate  :  [None, None]
conv_rate_plus_val1  :  [None, None]
conv_rate_plus_val2  :  [None, None]
driver_id  :  [1001, 1002]

--- Online features retrieved (using feature service v3, which uses a feature view with a push source---
Handling connection for 8082
acc_rate  :  [None, None]
avg_daily_trips  :  [None, None]
conv_rate  :  [None, None]
conv_rate_plus_val1  :  [None, None]
conv_rate_plus_val2  :  [None, None]
driver_id  :  [1001, 1002]

--- Simulate a stream event ingestion of the hourly stats df ---
An error occurred: 


 **Note:**
Currently, remote materialization not available in Feast when using the Remote Client so the above error showing in that step.

 Workaround: Consider using running it from pod like
  
 `kubectl exec deploy/feast-sample-kubernetes-auth -itc online -- bash -c 'feast materialize-incremental $(date -u +"%Y-%m-%dT%H:%M:%S")`


Terminate the process

In [25]:
for name, process in port_forward_processes.items():
    process.terminate()
    print(f"Stopped port forwarding for {name}")

Stopped port forwarding for offline_store
Stopped port forwarding for online_store
Stopped port forwarding for registry


Next :  Uninstall the Operator and all Feast objects -> 03-uninstall.ipynb